In [22]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Document Image Analysis/Assignment_3')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## First is to having a gray-scale image

In [23]:
import numpy as np
from PIL import Image

def rgb_to_grayscale(image):
    if image.dtype != np.float32:
        image = image.astype(np.float32)
    grayscale = 0.2989 * image[:, :, 0] + 0.5870 * image[:, :, 1] + 0.1140 * image[:, :, 2]
    return np.round(grayscale).astype(np.uint8)

run function here:

In [28]:
# here image_path is a list, you can set it like:
image_path = ['1_600x900.jpg', '2_1000x1500.jpg', '3_3700x5500.jpg']
# I'll only implement algorithm on 2_1000x1500.jpg in this notebook.
image_path = ['2_1000x1500.jpg']

def run_func(image_path = ['2_1000x1500.jpg'], output_path = []):
  for i in range(len(image_path)):
      image = Image.open(image_path[i])
      image_np = np.array(image)
      grayscale_image_np = rgb_to_grayscale(image_np)
      grayscale_image = Image.fromarray(grayscale_image_np)
      grayscale_image.save(output_path[i])
      return grayscale_image_np

def save_func(image_np, output_path):
  for i in range(len(output_path)):
      image_np = Image.fromarray(image_np)
      image_np.save(output_path[i])

In [29]:
grayscale_image_np = run_func(output_path = ['2_1000x1500_gray.jpg'])

## (a) Conceive an algorithm to perform a binarization based on a simple non adaptative thresholding.

In [30]:
def simple_binarize_image(grayscale_image_np, threshold=127):
    binary_image_np = (grayscale_image_np >= threshold) * 255
    return binary_image_np.astype(np.uint8)

In [31]:
threshold_value = 127

binary_image_path = ['2_1000x1500_simple_binary.jpg']
binary_image_np = simple_binarize_image(grayscale_image_np, threshold_value)
save_func(binary_image_np, binary_image_path)

## (b) Based on a global thresholding, conceive an other binarization algorithm.

In [ ]:
## here I choose otsu's method
def otsu_threshold(grayscale_image):
    hist = np.histogram(grayscale_image, bins=np.arange(257))[0]
    total = grayscale_image.size
    # Calculate the total grayscale value, using grayscale * num of pixel
    sum_all = np.dot(np.arange(256), hist)

    # Ajust threshold from i = 0 to 255
    sum_back = 0
    weight_back = 0
    max_variance = 0
    threshold = 0

    for i in range(256):
        weight_back += hist[i]
        if weight_back == 0:
            continue
        weight_fore = total - weight_back
        if weight_fore == 0:
            break

        sum_back += i * hist[i]
        mean_back = sum_back / weight_back
        mean_fore = (sum_all - sum_back) / weight_fore

        # Calculate between class variance
        variance = weight_back * weight_fore * (mean_back - mean_fore) ** 2

        # Maximize the distance between two
        if variance > max_variance:
            max_variance = variance
            threshold = i

    binary_image = np.where(grayscale_image > threshold, 255, 0).astype(np.uint8)
    return binary_image, threshold

In [32]:
binary_image_np, threshold = otsu_threshold(grayscale_image_np)
print('otsu_threshold:', threshold)
binary_image_path = ['2_1000x1500_binary_otsu.jpg']
save_func(binary_image_np, binary_image_path)

otsu_threshold: 153


## (c) Design an algorithm to perform binarisation based on local adaptative thresholding.

In [33]:
import numpy as np
from scipy.ndimage import uniform_filter

# Niblack's binarization method
def niblacks_method(grayscale_image, window_size=25, k=0.2):
    # Calculate the mean and standard deviation in the local neighborhood of each pixel
    mean = uniform_filter(grayscale_image, window_size, mode='constant')
    mean_sq = uniform_filter(grayscale_image**2, window_size, mode='constant')
    stddev = np.sqrt(mean_sq - mean**2)

    # Calculate the Niblack threshold
    threshold = mean - k * stddev

    # Apply the threshold
    binary_image = grayscale_image > threshold
    return binary_image.astype(np.uint8) * 255

In [34]:
binary_image_np = niblacks_method(grayscale_image_np)
binary_image_path = ['2_1000x1500_binary_niblack.jpg']
save_func(binary_image_np, binary_image_path)

## Optional: conceive and apply a fourth binarization algorithm of your choice.

In [35]:
## here I choose Bernsen's method

import numpy as np
from skimage.util import view_as_windows

def bernsen_local_thresholding(grayscale_image, window_size=15):
    window_shape = (window_size, window_size)
    local_windows = view_as_windows(grayscale_image, window_shape, step=1)

    # Calculate local min and max for each window
    local_mins = np.min(local_windows, axis=(2, 3))
    local_maxs = np.max(local_windows, axis=(2, 3))

    thresholds = (local_mins + local_maxs) / 2
    binary_image = np.zeros_like(grayscale_image, dtype=np.uint8)
    offsets = window_size // 2

    for i in range(offsets, offsets + thresholds.shape[0]):
        for j in range(offsets, offsets + thresholds.shape[1]):
            if grayscale_image[i, j] > thresholds[i - offsets, j - offsets]:
                binary_image[i, j] = 255
            else:
                binary_image[i, j] = 0

    return binary_image

In [37]:
binary_image_np = bernsen_local_thresholding(grayscale_image_np, window_size=3)
binary_image_path = ['2_1000x1500_binary_bernsen.jpg']
save_func(binary_image_np, binary_image_path)